In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
from ast import literal_eval
from tqdm import tqdm
from sklearn.metrics import classification_report
tqdm.pandas()
import random
random.seed(0)
from belt_nlp.bert_with_pooling import BertClassifierWithPooling

# Example - Model BERT with pooling

In this notebook we will show how to use basic methods `fit` and `predict` for the BERT model with pooling.

In [2]:
!export PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:128
!export PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True

In [3]:
raw_data_path = '../data/raw/'
processed_data_path = '../data/processed/'

file_format_users_filtered = processed_data_path + 'r3_{target}_{split}_users_scored_Timeline.csv' 
file_format_tmt_filtered = processed_data_path + '{split}_r3_{target}_top_mentioned_timelines_scored_Texts.csv'

In [ ]:
for target in target_list:
    
    for n_comment in [10,15,20, 5]:
        
        
        
        

In [4]:
target = 'ig'
n_comments = 10
text_col = 'Timeline'

# read data
train = pd.read_csv(
    file_format_users_filtered.format(target = target, split = "train"), 
    sep = ';', 
    encoding='utf-8-sig'
    )

train[f'comments_and_scores_{text_col}'] = train[f'comments_and_scores_{text_col}'].progress_apply(lambda x: literal_eval(x))

train[text_col] = train[f'comments_and_scores_{text_col}'].progress_apply(
    lambda x: " # ".join([comment for score, comment in x[-n_comments:]])
    ) 
train.Polarity = train.Polarity.map({
    "against": 0,
    "for": 1
})
test = pd.read_csv(
    file_format_users_filtered.format(target = target, split = "test"), 
    sep = ';', 
    encoding='utf-8-sig'
    )

test[f'comments_and_scores_{text_col}'] = test[f'comments_and_scores_{text_col}'].progress_apply(lambda x: literal_eval(x))

test[text_col] = test[f'comments_and_scores_{text_col}'].progress_apply(
    lambda x: " # ".join([comment for score, comment in x[-n_comments:]])
    ) 

test.Polarity = test.Polarity.map({
    "against": 0,
    "for": 1
})

100%|██████████| 599/599 [00:00<00:00, 317782.46it/s]


## Divide to train and test sets

In [5]:
X_train = train[text_col].tolist()
X_test = test[text_col].tolist()

y_train = train["Polarity"].tolist()
y_test = test["Polarity"].tolist()

## Fit the model

In [6]:
bert_model_name = 'pablocosta/bertabaporu-base-uncased'
epochs = 3
batch_size = 1

In [7]:
MODEL_PARAMS = {
    "batch_size": batch_size,
    "learning_rate": 5e-5,
    "epochs": epochs,
    "chunk_size": 510,
    "stride": 510,
    "minimal_chunk_length": 510,
    "pooling_strategy": "mean",
    "pretrained_model_name_or_path": 'pablocosta/bertabaporu-base-uncased'
}
model = BertClassifierWithPooling(**MODEL_PARAMS, device="cuda:0")

In [8]:
torch.cuda.empty_cache()


model.fit(X_train, y_train, epochs=epochs)  #  Warning about tokeninizing too long text is expected

In [9]:
y_pred = model.predict_classes(X_test)

In [10]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.69      0.72      0.70       339
           1       0.61      0.57      0.59       260

    accuracy                           0.66       599
   macro avg       0.65      0.65      0.65       599
weighted avg       0.65      0.66      0.65       599

